# Modifications de résultats des sondages

## Import préliminaires des librairies

In [126]:
import re
import numpy as np
import pandas as pd
import locale 
from locale import atof
# Régression linéaire
from sklearn.linear_model import LinearRegression

# Read files
import PyPDF2
from pdfminer.high_level import extract_text

In [ ]:
fp="rapport/2021-03-03.pdf"
text = extract_text(fp,page_numbers=[10])
question1="Êtes-vous  satisfait  ou  mécontent  d"
question2="Êtes-vous satisfait ou mécontent d"
# print(text.find(question1))
# print(text.find(question2))
# print(len(question2))
print(re.sub('\.\.\. (?![\n])','...\n',text))

In [100]:
with open(fp,"rb") as pdf_file:
    pdf_reader= PyPDF2.PdfFileReader(pdf_file)
    isEncrypted = pdf_reader.getIsEncrypted()
    nbPages= pdf_reader.numPages

    # Define columns
    page_content=pdf_reader.getPage(11).extractText()
page_content

"12\nL'Observatoire politique \n\nMars 2021\n04/03/2021\nDétail des résultats\nselon la préférence partisane et le vote à la présidentielle\n* Résultats à interpréter avec prudence compte\n-\ntenu des effectifs de répondants\nEn %\nTotal\nTout à fait \nconfiance\nPlutôt confiance\nPas vraiment \nconfiance\nPas confiance du \ntout\nSans opinion\nTOTAL\nCONFIANCE\nTOTAL\nPAS CONFIANCE\nEnsemble\n100\n6\n28\n29\n31\n6\n34\n60\nPREFERENCE PARTISANE\nTOTAL GAUCHE\n100\n5\n23\n34\n35\n3\n28\n69\nLa France Insoumise\n100\n3\n13\n30\n52\n2\n16\n82\nEurope Ecologie \n-\nLes Verts\n100\n6\n17\n45\n26\n6\n23\n71\nParti Socialiste / \nGénération.s\n/ PRG\n100\n4\n41\n34\n20\n1\n45\n54\nTOTAL EN MARCHE / MODEM\n100\n22\n71\n3\n4\n93\n7\nLa République en Marche !\n100\n26\n71\n0\n3\n97\n3\nTOTAL DROITE\n100\n3\n42\n27\n26\n2\n45\n53\nLes Républicains\n100\n2\n48\n27\n21\n2\n50\n48\nLe Rassemblement National\n100\n1\n7\n31\n59\n2\n8\n90\nSans préférence partisane\n100\n3\n23\n33\n29\n12\n26\n62\nVOTE

# Extraction des données avec PyPDF2
PyPDF2 semble être constant dans la forme d'extraction des données de Elab.
La structure des données ainsi que les détails d'extraction sont précisés ci-dessous:
* Elabe fourni les données détaillées pour deux personnalités que sont le président de la République et le premier ministre. Les noms n'étant pas indiqués sur la page de données, il faut les extraire ailleurs ou les préciser à la main.
* Il y a deux pages successives de données par individus
* Les catégories principales sont indiquées en capitales
* Il existe des catégories secondaires et tertiaires sans différenciation possible. Elles seront traités comme sous catégories.
* La base de chaque catégories n'est pas précisée, il faudra donc utiliser de la régression linéaire pour tenter de les retrouver
* Les trous dans les données ne sont pas visible. Il faut un système de vérification.
* Les variations ne sont pas données

__Format des données__
* Les données sont fournis en lisant les lignes dans l'ordre
* Chaque cellule est extraite par PyPDF2 une à la fois en introduisant un saut de ligne
* Les blancs dans les données ne créént pas de fusion d'information mais disparaissent sans faire de double saut de lignes
* Le caractère '__-__' compromet les cellules et induit un ou plusieurs sauts malencontreux :
    * Si le '-' apparait au milieu d'un texte, il induit un _double saut de ligne_ (ex: 18-24 ans)
    * Si le '-' apparait à la fin d'un texte il induit un _seul_ saut de ligne supplémentaire avant sont apparition (CSP-)  
* Il faudra donc préformater le texte au préalable


## Extract content (to delete later)

In [118]:
def get_page_content(num_page,fp)->str:
    with open(fp,"rb") as pdf_file:
        pdf_reader= PyPDF2.PdfFileReader(pdf_file)
        isEncrypted = pdf_reader.getIsEncrypted()
        nbPages= pdf_reader.numPages
        return pdf_reader.getPage(num_page-1).extractText()

    # Define columns
page_content=get_page_content(2,fp)
page_content

"Interrogation\nFiche technique\n2\n\nMode de recueil et dates de terrain\nLa\nreprésentativité\nde\n\na\nété\nassurée\nselon\nla\nméthode\ndes\nquotas\nappliquée\naux\nvariables\nsuivantes\n:\nsexe,\nâge\net\nprofession\nde\n\naprès\nstratification\npar\nrégion\net\ncatégorie\n\n.\nEchantillon de \n1 003  \npersonnes représentatif de la population française âgée de 18 ans et plus.\nInterrogation par Internet les \n2 et 3 mars 2021\n.\n04/03/2021\nL'Observatoire politique \n\nMars 2021\n"

## Préformatage

In [ ]:
##Préformatage
def preformatage(page_content)->str:
    #Remplacement des données d'age
    page_content=page_content.replace('18\n-\n24 ans','18-24 ans')
    page_content=page_content.replace('25\n-\n34 ans','25-34 ans')
    page_content=page_content.replace('35\n-\n49 ans','35-49 ans')
    page_content=page_content.replace('50\n-\n64 ans','50-64 ans')
    # Remplacement CSP-
    page_content=page_content.replace('CSP\n-','CSP-')
    # Remplacement EELV
    page_content=page_content.replace('Europe Ecologie \n-\nLes Verts','Europe Ecologie Les Verts')
    # Noms de candidats
    page_content=page_content.replace('Jean\n-\n','Jean-')
    # Remplacement /
    page_content=page_content.replace('/ \n','/ ')
    page_content=page_content.replace('\n/','/')
    # Remplacement Total .. en minuscule
    page_content=page_content.replace('TOTAL GAUCHE','Total Gauche')
    page_content=page_content.replace('TOTAL EN MARCHE / MODEM','Total en Marche / MODEM')
    page_content=page_content.replace('TOTAL DROITE','Total Droite')
    # Suppression fin de texte
    page_content=page_content.replace('\nLa cote de confiance du Président de la République','')
    page_content=page_content.replace('\nLa cote de confiance du Premier ministre','')
    # Suppression des notes de bas de pages
    page_content=page_content.replace(' (*)','')
    page_content=page_content.replace('(*)','')
    # Suppression du dernier saut de ligne
    page_content=page_content.rstrip('\n')
    return page_content

preformatage(page_content)


## Extraction de la base

In [121]:
def extract_base(fp,num_page=2)->int:
    page_content=get_page_content(num_page,fp)
    text_deb='Echantillon de \n'
    text_fin='\npersonnes'
    i_deb=page_content.find(text_deb)+len(text_deb)
    i_fin=page_content.find(text_fin)
    base=int(page_content[i_deb:i_fin].rstrip().replace(' ',''))
    return base
extract_base(fp)

1003

## Création du modèle de Dataframe

In [87]:
df=pd.DataFrame(columns=['Nom','Base','Date','Categorie','Groupe','tres_positif','positif','negatif','tres_negatif','nsp','total_positif','total_negatif'])

## Extraction de page formattée

In [114]:
def extract_series(data,i,columns,nom,base,date,prv_categorie)->pd.Series:
    """Extrait une serie de data à partir de l'indice i jusqu'à la prochaine valeur non digit

    Args:
        data (list): Toutes les données dans une liste 
        i (int): Indice courant dans les données
        columns (list): Liste des noms de colonnes 
        nom (str): Nom de la personnalité
        base (int): Nombre de sondés
        date (datetime.date): Date du sondage

    Returns:
        pd.Series: La serie contenant les valeurs à stocker
        int: new index
    """
    values=[nom,base,date]
    #Catégorie
    if data[i].isupper() and '+' not in data[i] and '-' not in data[i]:
        values.append(data[i])
        i+=1
    else:
        values.append(prv_categorie)
    #Groupe
    values.append(data[i])
    # On saut la cellule qui contient le 100% et qui ne sert a rien
    i+=2
    while i<len(data) and re.match('^\d+$',data[i]):
        values.append(float(data[i]))
        i+=1
    # Vérification
    while len(values)!=len(columns):
        print(values)
        j=int(input('Où faut-il ajouter un zéro ? (1-7)'))+4
        values=values[:j]+[0.0]+values[j:]
    return pd.Series(values,index=columns),i



def extract_detailled_page(page_content,df,nom,base,date)->pd.DataFrame:
    """
    Extrait une page du pdf et l'ajout au dataframe.
    
    Parameters
    ----------
    page_content: texte formatté de la page dot l'on souhaite extraire les données
    df: Dataframe pour stocker les données
    nom: Nom de la personnalité 
    base: Base d'étude (à compléter par regression plus tard)
    date: Date du sondage

    Returns
    -------
    Dataframe
        Dataframe df avec les nouvelles données
    """
    indice_debut=page_content.find('Ensemble')
    data=page_content[indice_debut:].split('\n')
    i=0
    cat='Ensemble'
    while i < len(data):
        if not re.match('^\d+$',data[i]):
            s,i=extract_series(data,i,df.columns,nom,base,date,cat)
            cat=s.Categorie
            df=df.append(s,ignore_index=True)
    return df
            

## Extraction pour une personnalité

In [117]:
def extract_detailled_perso(df,fp,page_indexes,nom,base,date)->pd.DataFrame:
    """Extrait les données sur toutes les pages pour une personnalité

    Args:
        df (pandas.Dataframe): Dataframe stockant les données
        fp (str): filepath du fichier à lire
        page_indexes (list): Liste des pages à extraire
        nom (str): Nom de la personnalité
        base (int): Nb de sondés
        date (datetime.date): Date du sondate
        inline (bool): Si True modification en ligne

    Returns:
        pd.DataFrame: Dataframe étendu des nouvelles données
    """
    for i in page_indexes:
        page_content=preformatage(get_page_content(i,fp))
        df=extract_detailled_page(page_content,df,nom,base,date)
    df=df.drop_duplicates(ignore_index=True)
    return df

extract_detailled_perso(df,fp,[11,12],'manu',1000,'date')
        

['manu', 1000, 'date', 'PROFESSION DU REPONDANT', 'Artisan, commerçant', 8.0, 34.0, 29.0, 29.0, 42.0, 58.0]
['manu', 1000, 'date', 'PREFERENCE PARTISANE', 'Total en Marche / MODEM', 22.0, 71.0, 3.0, 4.0, 93.0, 7.0]
['manu', 1000, 'date', 'PREFERENCE PARTISANE', 'La République en Marche !', 26.0, 71.0, 0.0, 3.0, 97.0, 3.0]
['manu', 1000, 'date', 'VOTE PRESIDENTIELLE 2017 (1ER TOUR)', 'Benoît HAMON', 3.0, 32.0, 31.0, 34.0, 35.0, 65.0]


,Nom,Base,Date,Categorie,Groupe,tres_positif,positif,negatif,tres_negatif,nsp,total_positif,total_negatif
0,manu,1000,date,Ensemble,Ensemble,6.0,28.0,29.0,31.0,6.0,34.0,60.0
1,manu,1000,date,SEXE DU REPONDANT,Homme,8.0,31.0,26.0,32.0,3.0,39.0,58.0
2,manu,1000,date,SEXE DU REPONDANT,Femme,4.0,26.0,32.0,30.0,8.0,30.0,62.0
3,manu,1000,date,AGE DU REPONDANT,18-24 ans,11.0,26.0,35.0,19.0,9.0,37.0,54.0
4,manu,1000,date,AGE DU REPONDANT,25-34 ans,5.0,28.0,37.0,23.0,7.0,33.0,60.0
5,manu,1000,date,AGE DU REPONDANT,35-49 ans,5.0,25.0,27.0,37.0,6.0,30.0,64.0
6,manu,1000,date,AGE DU REPONDANT,50-64 ans,6.0,30.0,26.0,33.0,5.0,36.0,59.0
7,manu,1000,date,AGE DU REPONDANT,65 ans et plus,4.0,32.0,27.0,33.0,4.0,36.0,60.0
8,manu,1000,date,PROFESSION DU REPONDANT,Actifs,5.0,29.0,29.0,31.0,6.0,34.0,60.0
9,manu,1000,date,PROFESSION DU REPONDANT,"Artisan, commerçant",8.0,34.0,29.0,29.0,0.0,42.0,58.0


In [125]:
df=pd.DataFrame(columns=['Nom','Base','Date','Categorie','Groupe','tres_positif','positif','negatif','tres_negatif','nsp','total_positif','total_negatif'])
date='2020-06-03'
filepath='rapport/{}.pdf'.format(date)
president='Emmanuel MACRON'
page_president=[10,11]
ministe='Edouard PHILIPPE'
page_ministre=[13,14]
base=extract_base(filepath)
df=extract_detailled_perso(df,filepath,page_president,president,base,date)
df=extract_detailled_perso(df,filepath,page_ministre,ministe,base,date)
df

['Emmanuel MACRON', 1002, '2020-06-03', 'PREFERENCE PARTISANE', 'La République en Marche !', 31.0, 65.0, 4.0, 1.0, 96.0, 4.0]
['Emmanuel MACRON', 1002, '2020-06-03', 'PREFERENCE PARTISANE', 'Les Républicains', 6.0, 36.0, 35.0, 23.0, 42.0, 58.0]
['Emmanuel MACRON', 1002, '2020-06-03', 'VOTE PRESIDENTIELLE 2017 (1ER TOUR)', 'Benoît HAMON', 4.0, 30.0, 32.0, 34.0, 34.0, 66.0]
['Emmanuel MACRON', 1002, '2020-06-03', 'VOTE PRESIDENTIELLE 2017 (1ER TOUR)', 'François FILLON', 10.0, 36.0, 25.0, 29.0, 46.0, 54.0]
['Edouard PHILIPPE', 1002, '2020-06-03', 'PROFESSION DU REPONDANT', 'Cadre, profession intellectuelle supérieure', 13.0, 39.0, 26.0, 22.0, 52.0, 48.0]
['Edouard PHILIPPE', 1002, '2020-06-03', 'PREFERENCE PARTISANE', 'Le Rassemblement National', 14.0, 20.0, 62.0, 3.0, 14.0, 83.0]


,Nom,Base,Date,Categorie,Groupe,tres_positif,positif,negatif,tres_negatif,nsp,total_positif,total_negatif
0,Emmanuel MACRON,1002,2020-06-03,Ensemble,Ensemble,7.0,26.0,28.0,34.0,5.0,33.0,62.0
1,Emmanuel MACRON,1002,2020-06-03,SEXE DU REPONDANT,Homme,8.0,28.0,25.0,34.0,5.0,36.0,59.0
2,Emmanuel MACRON,1002,2020-06-03,SEXE DU REPONDANT,Femme,6.0,23.0,31.0,34.0,6.0,29.0,65.0
3,Emmanuel MACRON,1002,2020-06-03,AGE DU REPONDANT,18-24 ans,11.0,24.0,31.0,22.0,12.0,35.0,53.0
4,Emmanuel MACRON,1002,2020-06-03,AGE DU REPONDANT,25-34 ans,11.0,30.0,22.0,32.0,5.0,41.0,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...
83,Edouard PHILIPPE,1002,2020-06-03,VOTE PRESIDENTIELLE 2017 (1ER TOUR),Marine LE PEN,1.0,17.0,25.0,54.0,3.0,18.0,79.0
84,Edouard PHILIPPE,1002,2020-06-03,VOTE PRESIDENTIELLE 2017 (1ER TOUR),"Abstention, vote blanc ou nul",5.0,23.0,35.0,27.0,10.0,28.0,62.0
85,Edouard PHILIPPE,1002,2020-06-03,VOTE PRESIDENTIELLE 2017 (2ND TOUR),Emmanuel MACRON,18.0,45.0,24.0,9.0,4.0,63.0,33.0
86,Edouard PHILIPPE,1002,2020-06-03,VOTE PRESIDENTIELLE 2017 (2ND TOUR),Marine LE PEN,2.0,16.0,24.0,55.0,3.0,18.0,79.0


In [ ]:
def correct_base(df,assoc)->pd.Dataframe:
    """Calcule les bases possibles en utilisant la régression linéaire

    Args:
        df (pandas.Dataframe): Dataframe contenant les données à évaluer dans la colonne Base
        assoc (dict): Dictionnaire contenant les associations de la forme cle-> liste de valeurs où la clef représente un ensemble et les valeurs des sous ensembles 

    Returns:
        pd.Dataframe: le dataframe modifié
    """
    lr = LinearRegression()
    


In [ ]:

# with open("rapport/2020-10-14.pdf","rb") as pdf_file:
#     pdf_report= PyPDF2.PdfFileReader(pdf_file)
#     isEncrypted = pdf_report.getIsEncrypted()
#     nbPages= pdf_report.numPages

#     page1=pdf_report.getPage(15)
#     text=page1.extractText().replace(',','.')
#     text = extract_text(pdf_file,page_numbers=[14])

# print(text)

print(df.isnull().values.any())
df.head(10)
df.describe()
df[df.Base==0]

## Find mistakes and try to correct them

In [ ]:
def find_fav_mistake(df):
    tfav=df['Très favorable']
    pfav=df['Plutôt favorable']
    stfav=df['ST Favorable']
    fav_df= df[np.abs(tfav+pfav-stfav)>0.11]
    return fav_df

def find_def_mistake(df):
    tdef=df['Très défavorable']
    pdef=df['Plutôt défavorable']
    stdef=df['ST Défavorable']
    def_df= df[np.abs(tdef+pdef-stdef)>0.11]
    return def_df

def find_nsp_mistake(df):
    stfav=df['ST Favorable']
    stdef=df['ST Défavorable']
    nsp=df['Nsp']
    tot_df= df[np.abs(stfav+stdef+nsp-100)>0.11]
    return tot_df

def find_mistake(df):
    fav_df= find_fav_mistake(df)
    def_df= find_def_mistake(df)
    tot_df=find_nsp_mistake(df)
    error_df=pd.concat([fav_df,def_df,tot_df])
    return error_df

In [ ]:
def realign_fav(df,i=0,imax=-1):
    df_err=find_fav_mistake(df)
    zeros_cols= [x for x in ["Très favorable","Plutôt favorable","ST Favorable"] if df[x].all()==0]
    if imax==-1:
        imax=len(zeros_cols)*len(df)
    if not df_err.empty and i<=imax:
        col=zeros_cols[0]
        values=df_err[col].tolist()
        values=values[1:]+[values[0]]
        df.loc[df_err.index,col]=values
        df=realign_fav(df,i+1)
    return df

def realign_def(df,i=0,imax=-1):
    df_err=find_fav_mistake(df)
    zeros_cols= [x for x in ["Très défavorable","Plutôt défavorable","ST Défavorable"] if df[x].all()==0]
    if imax==-1:
        imax=len(zeros_cols)*len(df)
    if not df_err.empty and i<=imax:
        col=zeros_cols[0]
        values=df_err[col].tolist()
        values=values[1:]+[values[0]]
        df.loc[df_err.index,col]=values
        df=realign_def(df,i+1)
    return df

def realign_nsp(df,i=0,imax=-1):
    df_err=find_fav_mistake(df)
    zeros_cols= [x for x in ["ST Favorable","ST Défavorable","Nsp"] if df[x].all()==0]
    if imax==-1:
        imax=len(zeros_cols)*len(df)
    if not df_err.empty and i<=imax:
        col=zeros_cols[0]
        values=df_err[col].tolist()
        values=values[1:]+[values[0]]
        df.loc[df_err.index,col]=values
        df=realign_nsp(df,i+1)
    return df

    



In [ ]:
def correct_mistakes(df):
    df_err=find_fav_mistake(df)
    for nom in df_err.Nom.unique():
        for cat in df_err[df_err.Nom==nom].Categorie.unique():
            cond=((df_err.Nom==nom) & (df_err.Categorie==cat))
            df_err.loc[cond]=realign_fav(df_err[cond])
    df.loc[df_err.index]=df_err

    df_err=find_def_mistake(df)
    for nom in df_err.Nom.unique():
        for cat in df_err[df_err.Nom==nom].Categorie.unique():
            cond=((df_err.Nom==nom) & (df_err.Categorie==cat))
            df_err.loc[cond]=realign_def(df_err[cond])
    df.loc[df_err.index]=df_err

    df_err=find_nsp_mistake(df)
    for nom in df_err.Nom.unique():
        for cat in df_err[df_err.Nom==nom].Categorie.unique():
            cond=((df_err.Nom==nom) & (df_err.Categorie==cat))
            df_err.loc[cond]=realign_nsp(df_err[cond])
    df.loc[df_err.index]=df_err
    return df

In [ ]:
df_err=find_mistake(df)
if len(df_err)>0:
    index=df_err.iloc[0:1].index[0]
    val=df_err.iloc[0].values[3:]
    print(val)
    correct=[]
    cols=["Base","Très favorable","Plutôt favorable","ST Favorable","Plutôt défavorable","Très défavorable","ST Défavorable","Nsp","Evolution ST Favorable","Evolution ST Défavorable"]
    while len(correct)!=10:
        correct=input()
        correct=[float(x) for x in correct.split(' ')]
    print(correct)
    df.loc[index,cols]=correct

In [ ]:
df_err=find_mistake(df)
df_err

In [ ]:

# df.to_pickle('data/'+filename+'.p')